# Adiabatic SSM model

## 2. Scattered interpolation

This notebook takes multiple local SSM models which have been identified at pre-tensioned equilibria and interpolates them using scattered interpolation to obtain one adiabatic SSM model. The model performance is evaluated by its ability to predict open-loop trajectories.

In [1]:
from os import listdir
from os.path import join
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [2]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [3]:
%matplotlib qt
import matplotlib.pyplot as plt

In [40]:
# directory containing the SSM models
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic"
# model_names = sorted(listdir(model_dir))
# model_names = ["north","west", "origin", "east", "south"]
model_names = ["northwest", "north", "northeast", "west", "origin", "east", "southwest", "south", "southeast"] # ["north","west", "origin", "east", "south"] # 
print(model_names)
N_models = len(model_names)

['northwest', 'north', 'northeast', 'west', 'origin', 'east', 'southwest', 'south', 'southeast']


Load the pre-tensioned equilibrium of each model and plot adiabatic SSM map

In [41]:
q_eq, u_eq = [], []
for model in model_names:
    with open(join(model_dir, model, "SSMmodel", "pre-tensioned_rest_q.pkl"), "rb") as f:
        q_eq.append(pickle.load(f))
    with open(join(model_dir, model, "decay", "info.yaml"), "r") as f:
        info = yaml.safe_load(f)
    u_eq.append(np.array(info['pre_tensioning']))
print(q_eq)
print(u_eq)
xy_rest = np.array([q_eq[i][:2] for i in range(N_models)])

[array([-34.87879219,  35.84572228, -10.60406392]), array([ 0.54467017, 35.57246381, -6.34727727]), array([ 35.552736  ,  34.86109908, -14.7558155 ]), array([-35.30795769,  -0.42439489,  -4.08499071]), array([ 0.03909428, -0.08913968, -0.00175868]), array([34.95441827, -0.27574755, -7.94720868]), array([-35.09507886, -37.14043854, -10.94062807]), array([  0.27917007, -35.77321019,  -6.25407621]), array([ 35.12892494, -35.8256824 , -14.73033781])]
[array([200, 200,   0,   0, 200, 200,   0,   0]), array([200,   0,   0,   0, 200,   0,   0,   0]), array([200,   0,   0, 200, 200,   0,   0, 200]), array([  0, 200,   0,   0,   0, 200,   0,   0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([  0,   0,   0, 200,   0,   0,   0, 200]), array([  0, 200, 200,   0,   0, 200, 200,   0]), array([  0,   0, 200,   0,   0,   0, 200,   0]), array([  0,   0, 200, 200,   0,   0, 200, 200])]


In [42]:
models = []
for model_name in model_names:
    with open(join(model_dir, model_name, "SSMmodel", "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
Vde = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B = [model['model']['B'] for model in models]
ROMOrder = models[0]['params']['ROM_order']
SSMOrder = models[0]['params']['SSM_order']
for model in models:
    assert model['params']['ROM_order'] == ROMOrder
    assert model['params']['SSM_order'] == SSMOrder

Compute interpolations: Delaunay triangulation, Barycentric linear interpolation

In [62]:
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.spatial import Delaunay

interpolator = CloughTocher2DInterpolator

# xy_rest[0][0] -= 2

# compute delauny triangulation attached to the pre-tensioned equilibria
tri = Delaunay(xy_rest)
# create interpolants for the different coefficient matrices
interpolation = {}
for name, coeffs in [('W', w_coeff), ('V', Vde), ('R', r_coeff), ('B', B), ('u_bar', u_eq), ('q_bar', q_eq)]:
    interpolation[name] = interpolator(tri, coeffs)

Visualize the interpolation scheme

In [63]:
fig, ax = plt.subplots(1, 1, dpi=150)
ax.triplot(xy_rest[:,0], xy_rest[:,1], tri.simplices.copy(), color="black", ls="-", lw=0.8)
for i in range(N_models):
    ax.plot(xy_rest[i,0], xy_rest[i,1], marker="D", markersize=8, ls="", color="tab:blue")
ax.set_aspect("equal")
ax.set_xlabel(r"$x$ [mm]")
ax.set_ylabel(r"$y$ [mm]")
# fig.suptitle("Delaunay triangulation")
plt.show()

In [64]:
nx, ny = (100, 100)
grid_lims = (-40, 40)
x = np.linspace(grid_lims[0], grid_lims[1], nx)
y = np.linspace(grid_lims[0], grid_lims[1], ny)
# xv, yv = np.meshgrid(x, y, sparse=False)

model_weights = np.zeros((N_models, nx, ny))
for i in range(nx):
    for j in range(ny):
        p = [x[i], y[j]]
        simplex = tri.find_simplex(p)
        if simplex == -1:
            # extrapolation would be necessary
            model_weights[:, j, i] = np.nan
        else:
            b = tri.transform[simplex,:2] @ (p - tri.transform[simplex,2])
            c = np.r_[b, 1 - b.sum()]
            point_idx = tri.simplices[simplex]
            model_weights[point_idx, j, i] = c

In [65]:
model_weights.shape

(9, 100, 100)

In [66]:
fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=100, sharey=True, sharex=True, constrained_layout=True)
im = None
axs = axs.ravel()
for i, model in enumerate(model_names):
    ax = axs[i]
    im = ax.contourf(x, y, model_weights[i, :, :],
                    vmin=0, vmax=1, levels=100) # cmap=mpl.colormaps[colormaps[i]])
    # ax.set_title(model)
    ax.set_xlabel(r"$x$ [mm]")
    ax.set_aspect("equal")
for ax in axs[[0, 3, 6]]:
    ax.set_ylabel(r"$y$ [mm]")
fig.colorbar(im, ax=axs.ravel().tolist(), fraction=0.03, aspect=35)
plt.show()

In [67]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
data_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/origin"
rest_file = join(robot_dir, 'rest_qv.pkl')

Compute observables (delay embedding)

In [68]:
N_DELAY = 4
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Predict open-loop trajectories using the interpolated adiabatic SSM model

In [69]:
test_results = {}
test_trajectory_folders = ['open-loop_circle']
test_trajectories = []
for test_traj in test_trajectory_folders:
    traj_dir = join(robot_dir, "dataCollection", test_traj)
    (t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                      rest_file=rest_file,
                                      output_node=TIP_NODE, return_inputs=True, traj_index=0)
    # z = (z.T - q_eq).T
    # u = (u.T - u_eq).T
    y = assemble_observables(z)
    print("y.shape:", y.shape)
    print("u.shape:", u.shape)

    test_trajectories.append({
            'name': test_traj,
            't': t,
            'z': z,
            'u': u,
            'y': y
        })

q_rest: [ 1.18616278e+01  1.73091668e-02  2.75127778e+01 ... -4.83685413e+00  1.57972770e+00
  8.69078237e+01]
q_rest: [  3.54595786   0.30603326 195.61577293]
y.shape: (15, 1002)
u.shape: (8, 1002)


In [70]:
use_origin_only = False
origin_idx = model_names.index("origin")
def interpolate_coeffs(xy, coeff_name):
    if use_origin_only:
        if coeff_name == "W":
            return w_coeff[origin_idx]
        elif coeff_name == "R":
            return r_coeff[origin_idx]
        elif coeff_name == "V":
            return Vde[origin_idx]
        elif coeff_name == "B":
            return B[origin_idx]
        elif coeff_name == "u_bar":
            return np.zeros(8)
        elif coeff_name == "q_bar":
            return np.zeros(3)
        else:
            raise RuntimeError("No interpolation available for these coefficents: {coeff_name}")
    else:
        if coeff_name not in interpolation.keys():
            raise RuntimeError("No interpolation available for these coefficents: {coeff_name}")
        else:
            return interpolation[coeff_name](xy).squeeze()

In [71]:
def advect_adiabaticRD_with_inputs(t, y0, u, y, know_target=True):
    dt = 0.01
    N = len(t)-1
    x = np.full((6, N+1), np.nan)
    y_pred = np.full((15, N+1), np.nan)
    y_pred[:, 0] = y0
    y_bar = np.full((15, N+1), np.nan)
    u_bar = np.full((8, N+1), np.nan)
    xdot = np.full((6, N+1), np.nan)
    weights = np.full((N_models, N+1), np.nan)

    for i in range(N):
        # compute the weights used at this timestep
        try:
            simplex = tri.find_simplex(y[:2, i])
            b = tri.transform[simplex,:2] @ (y[:2, i] - tri.transform[simplex,2])
            c = np.r_[b, 1 - b.sum()]
            point_idx = tri.simplices[simplex]
            weights[point_idx, i] = c
            # assert np.all(c > 0)
            # compute and integrate the dynamics at this timestep
            y_bar[:, i] = np.tile(interpolate_coeffs(y[:2, i], 'q_bar'), 1 + N_DELAY)
            u_bar[:, i] = interpolate_coeffs(y[:2, i], 'u_bar')
            # x[i] = V[i]^T @ (y[i] - y_bar[i])
            # if i == 0:
            x[:, i] = interpolate_coeffs(y[:2, i], 'V').T @ (y_pred[:, i].T - y_bar[:, i]) # y[:, i]) # 
            # xdot[i] = R(x[i]) + B[i] @ (u[i] - u_bar[i])
            xdot[:, i] = (interpolate_coeffs(y[:2, i], 'R') @ utils.phi(x[:, i].reshape(-1, 1), ROMOrder)).flatten() + interpolate_coeffs(y[:2, i], 'B') @ (u[:, i] - u_bar[:, i]) # -0.5 * np.eye(6) @ x[:, i] # -0.001 * np.ones(6) # 
            # forward Euler: x[i+1] = x[i] + dt * xdot[i]
            x[:, i+1] = x[:, i] + dt * xdot[:, i]
            # y[i+1] = W(x[i+1]) + y_bar[i]
            y_pred[:, i+1] = (interpolate_coeffs(y[:2, i], 'W') @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder)).T + y_bar[:, i]
            if not know_target:
                y[:, i+1] = y_pred[:, i+1]
        except Exception as e:
            print("i =", i)
            print("y_pred[i-1]:", y_pred[:, i-1])
            print("y[i-1]:", y[:, i-1])
            print("x[i-1]:", x[:, i-1])
            print("=======================================")
            print("y_pred[i]:", y_pred[:, i])
            print("y[i]:", y[:, i])
            print("x[i]:", x[:, i])
            print("=======================================")
            print("y_pred[i+1]:", y_pred[:, i+1])
            print("y[i+1]:", y[:, i+1])
            print("x[i+1]:", x[:, i+1])
            print("=======================================")
            print(e)
            break
            # raise e
    return t, x, y_pred, xdot, y_bar, u_bar, weights

In [72]:
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

global t_prev
t_prev = 0

def advect_adiabaticRD_with_inputs_RK45(t, y0, u, y_target):
    uInterpFun = interp1d(t, u, axis=1, fill_value="extrapolate")
    yTargetInterpFun = interp1d(t, y_target, axis=1, fill_value="extrapolate")
    def R_t(t, x):
        global t_prev
        print("t:", t, ", t_prev:", t_prev)
        x = np.atleast_2d(x)
        y_target_prev = yTargetInterpFun(t_prev)
        y_target = yTargetInterpFun(t)
        y_bar_prev = np.tile(interpolate_coeffs(y_target_prev[:2], 'q_bar'), 1 + N_DELAY)
        y_bar = np.tile(interpolate_coeffs(y_target[:2], 'q_bar'), 1 + N_DELAY)
        y = ((interpolate_coeffs(y_target_prev[:2], 'W') @ utils.phi(x, SSMOrder)).T + y_bar_prev).T
        # x[i] = V[i]^T @ (y[i] - y_bar[i])
        x = interpolate_coeffs(y_target[:2], 'V').T @ (y.T - y_bar).T
        u_bar = interpolate_coeffs(y_target[:2], 'u_bar')
        xdot = ((interpolate_coeffs(y_target[:2], 'R') @ utils.phi(x, ROMOrder)).T + interpolate_coeffs(y_target[:2], 'B') @ (uInterpFun(t) - u_bar)).T
        # print(x)
        t_prev = t
        return xdot
    # solve IVP of reduced dynamics using open-loop inputs
    x0 = interpolate_coeffs(y0[:2], 'V').T @ (y0.T - np.tile(interpolate_coeffs(y0[:2], 'q_bar'), 1 + N_DELAY))
    print("x0:", x0)
    sol = solve_ivp(R_t,
                    t_span=[t[0], t[-1]],
                    t_eval=t,
                    y0=x0,
                    method="RK45",
                    vectorized=True,
                    rtol=1e-3,
                    atol=1e-3)
    # resulting (predicted) open-loop trajectory in reduced coordinates
    xTraj = sol.y
    yTraj = np.zeros_like(y_target)
    for i in range(xTraj.shape[1]):
        yTraj[:, i] = (interpolate_coeffs(y_target[:2, i], 'W') @ utils.phi(xTraj[:, i].reshape(-1, 1), SSMOrder)).T + np.tile(interpolate_coeffs(y_target[:2, i], 'q_bar'), 1 + N_DELAY)
    return t, yTraj

In [76]:
traj = test_trajectories[0]
print(traj['name'])
t, x, y_pred, xdot, y_bar, u_bar, weights = advect_adiabaticRD_with_inputs(traj['t'], traj['y'][:, 0], traj['u'], traj['y'], know_target=True)
# slice_traj = np.s_[:650]
# t, y_pred = advect_adiabaticRD_with_inputs_RK45(traj['t'][slice_traj], traj['y'][:, 0], traj['u'][:, slice_traj], y_target=traj['y'][:, slice_traj])

open-loop_circle
i = 305
y_pred[i-1]: [-4.33002154e+50 -5.31712472e+51  2.18258586e+50  2.29897836e+50  3.09366581e+51  7.82947144e+48
  4.24088688e+50  5.31096455e+51 -2.76018561e+49  1.92831235e+50  2.18493592e+51 -9.37043271e+49
 -4.29728571e+50 -5.43720922e+51 -2.68476709e+50]
y[i-1]: [-8.88180449 30.18302405 -5.29426655 -9.07629065 30.1329251  -5.28736087 -9.27058468 30.08178077
 -5.28061264 -9.46474833 30.02992768 -5.27412361 -9.65840454 29.97671544 -5.26743414]
x[i-1]: [ 8.92750974e+48  1.95187833e+48 -5.41978897e+48 -5.80868148e+48  3.17143950e+48 -2.23085916e+50]
y_pred[i]: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
y[i]: [-9.07629065 30.1329251  -5.28736087 -9.27058468 30.08178077 -5.28061264 -9.46474833 30.02992768
 -5.27412361 -9.65840454 29.97671544 -5.26743414 -9.85145638 29.92180465 -5.26043844]
x[i]: [nan nan nan nan nan nan]
y_pred[i+1]: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
y[i+1]: [ -9.27058468  30.08178077  -5.28061264  -9.

/home/jonas/miniconda3/envs/soft/lib/python3.8/site-packages/sklearn/preprocessing/_polynomial.py:440: RuntimeWarning: overflow encountered in multiply
  np.multiply(
/tmp/ipykernel_24913/4033144024.py:32: RuntimeWarning: invalid value encountered in matmul
  y_pred[:, i+1] = (interpolate_coeffs(y[:2, i], 'W') @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder)).T + y_bar[:, i]


In [78]:
plt.close('all')
# Trajectory xyz vs. t (predicted vs. actual)
axs = plot.traj_xyz_txyz(t,
                            y_pred[0, :], y_pred[1, :], y_pred[2, :],
                            show=False)
axs = plot.traj_xyz_txyz(traj['t'],
                        traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
                        color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
for ax in axs:
    ax.set_ylim(-35, 35)
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.tight_layout()
plt.show()

# Trajectory 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
                      show=False)
ax = plot.traj_2D_xy(traj['z'][0, :], traj['z'][1, :],
                        color="tab:orange", ax=ax, show=False)
ax.set_xlim(-35, 35)
ax.set_ylim(-35, 35)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

# weights
axs = plot.adiabatic_model_weights(t, weights, model_names)

# psi_bar
axs = plot.traj_xyz_txyz(t,
                        y_bar[0, :], y_bar[1, :], y_bar[2, :],
                        xyz_names=[r"$\bar{\psi}_1(t)$", r"$\bar{\psi}_2(t)$", r"$\bar{\psi}_3(t)$"],
                        show=False, rotate_yticks=True)
plt.suptitle(r"$\bar{\psi}(t)$")
plt.show()

# xi_dot
axs = plot.traj_xyz_txyz(t,
                        xdot[0, :], xdot[1, :], xdot[2, :],
                        xyz_names=[r"$\dot{\xi}_1(t)$", r"$\dot{\xi}_2(t)$", r"$\dot{\xi}_3(t)$"],
                        show=False, rotate_yticks=True)
# for ax in axs:
#     ax.set_ylim(-100, 100)
plt.suptitle(r"$\dot{\xi}(t)$")
plt.show()

# xi
axs = plot.traj_xyz_txyz(t,
                        x[0, :], x[1, :], x[2, :],
                        xyz_names=[r"$\xi_1(t)$", r"$\xi_2(t)$", r"$\xi_3(t)$"],
                        show=False, rotate_yticks=True)
# for ax in axs:
#     ax.set_ylim(-100, 100)
plt.suptitle(r"$\xi(t)$")
plt.show()
